# Retrieval Augemented Generation (RAG)

In the prevous two recepies, we learned how to [obtain an embedding](./10-embeddings.ipynb), as well as how to find the [similarity](./11-similarity-search.ipynb) between two embeddings. Retrievel Augmente Generation uses both of these to achieve the goal of querying an LLM and also providing some documents for additional context. For example, if you want the LLM to answer some questions based on specific documents you have, you can feed those documents in 

A RAG model is primarily made up of two components, the retriever, and the reader

In [1]:
from langchain.docstore.document import Document
from langchain_dartmouth.retrievers.document_compressors import DartmouthReranker
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader

<div class="alert alert-info">

**Note:** The `DirectoryLoader` function is a part of LangChain that accepts a directory, a regex expression, and a [loader class](./10-embeddings.ipynb). It's an easy way to load several documents that are in a directory at once
</div>

In [3]:
loader = DirectoryLoader('./rag_documents', glob="**/*.txt", loader_cls=TextLoader)
collection = loader.load()
collection

[Document(metadata={'source': 'rag_documents\\asteroids.txt'}, page_content="**Asteroids: The Mysterious and Ancient Building Blocks of Our Solar System**\n\nAsteroids, also known as minor planets or planetoids, are small, rocky objects that orbit the Sun. They are remnants from the early days of our solar system, and their study has provided valuable insights into the formation and evolution of the cosmos. These mysterious bodies have captivated the imagination of scientists and researchers for centuries, and their exploration continues to uncover new secrets about the universe.\n\n**Composition and Types of Asteroids**\n\nAsteroids are typically small, with diameters ranging from a few meters to hundreds of kilometers. They are composed of rock, metal, and ice, and are thought to be the remnants of the early solar system. There are two main types of asteroids: stony asteroids, which are composed mostly of silicate minerals, and metal asteroids, which are rich in iron and nickel.\n\nT

Given a **collection** of documents, we are interested in finding the ones most relevent to our question. Once a query is posed, the `compress_documents()` function from the `DartmouthReranker` class can be used, which uses the concept about [similarity](./11-similarity-search.ipynb), to return a list of documents which are most relevant to the query.

In [11]:
reranker = DartmouthReranker()

In [12]:
from langchain_dartmouth.llms import ChatDartmouth

query = "Are asteroids going to hit me?"
ranked_docs = reranker.compress_documents(query=query, documents=collection)

for doc in ranked_docs:
    print(doc.metadata['source'])

rag_documents\asteroids.txt
rag_documents\hot_sauce.txt
rag_documents\history.txt


<div class="alert alert-info">

**Note:** We can see that when our query is related to asteroids, the document correcly ranks the asteroids.txt as the most relevent document. Now, we can prepend the content of the first ranked document to our query to get a response which considers the information in the document
</div>

In [13]:
llm = ChatDartmouth(model_name="llama-3-1-8b-instruct")
response = llm.invoke(ranked_docs[0].page_content + 'Considering this, answer the following question:' +  query)
print(response)

content="The likelihood of an asteroid hitting you is extremely low. According to NASA, the chances of being hit by a large asteroid are extremely small. In fact, the chances of being struck by a meteorite (a small asteroid or fragment) are about 1 in 1,000,000.\n\nMost asteroids are small and burn up in the atmosphere before they can cause any damage. Even larger asteroids that are on a collision course with Earth are usually detected by astronomers and scientists years in advance, allowing for potential deflection or disruption of their trajectory.\n\nNASA and other space agencies around the world are actively monitoring the skies for near-Earth asteroids (NEAs) and have developed various strategies to prevent or mitigate potential asteroid impacts. These strategies include:\n\n1. Detection: Using powerful telescopes and advanced technologies to detect and track NEAs.\n2. Tracking: Monitoring the orbits of NEAs to predict their paths and potential impacts.\n3. Deflection: Using vario

# Summary
In this recipe we use a query and `DartmouthReranker` to retrieve relative documents from a **collection**. The content of the highest ranked document is then prepended in the a prompt to an llm. This is an implementation of retreival augmented generation.